# mdx REST Client for Python コマンド一覧

この Notebook では、mdx REST API にアクセスするためのPythonクライアントライブラリで利用できる操作の一覧を掲載しています。

## 準備

1. mdx REST API 認証トークン設定
2. mdx VM にSSHログインするためのキーペア作成
3. mdx の REST API エンドポイントに接続できることの確認

### 利用するライブラリの読込

In [ ]:
# mdx_{ext,lib}.py ファイルのパスを取得
import os
import sys
import json

mdx_lib_path = os.path.join(os.path.dirname(os.path.abspath('__file__')), '../src')

# モジュール検索パスを追加
if mdx_lib_path not in sys.path:
    sys.path.append(mdx_lib_path)

from mdx_ext import MdxResourceExt

def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

### mdx REST API 認証トークン設定

- mdx REST API 認証トークンは、[mdxユーザポータル](https://oprpl.mdx.jp/) の「トークン発行」により取得する。
  * 「トークン発行」は、ユーザポータル画面右上にあるユーザ名をクリックすると表示される。

In [ ]:
# mdx REST API 認証トークン設定
from getpass import getpass
mdx_token = getpass()

### インスタンスの初期化

In [ ]:
mdx = MdxResourceExt(mdx_token)

### mdx REST API エンドポイント接続確認

- HTTPステータスコード 200 が返ることを確認する。

In [ ]:
# デフォルトのresolverがIPv6のアドレスを返すことにより以降のAPIで接続不可の場合があるため、以下のコードを実行しておく。
def use_ipv4_only():
    import socket
    old_getaddrinfo = socket.getaddrinfo
    def new_getaddrinfo(*args, **kwargs):
        responses = old_getaddrinfo(*args, **kwargs)
        return [response
                for response in responses
                if response[0] == socket.AF_INET]
    socket.getaddrinfo = new_getaddrinfo

use_ipv4_only()

In [ ]:
!curl https://oprpl.mdx.jp -w '%{http_code}\n' -o /dev/null -s

## プロジェクト関連

プロジェクト情報に関するライブラリ一覧

### プロジェクト情報取得  

自身が所属しているプロジェクトの情報一覧を取得します。

In [ ]:
import json
r = mdx.get_assigned_projects()
print(json.dumps(r[0]["projects"], indent=2))

### 操作対象プロジェクト指定  

ライブラリの多くはプロジェクトを指定した操作を行うものです。  
ライブラリを利用する際は、事前準備として、以下の関数で対象のライブラリを指定しておきます。

In [ ]:
project_name = 
mdx.set_current_project_by_name(project_name)

### プロジェクト指定の確認

操作対象プロジェクト指定で指定したプロジェクトの確認を行います。  

In [ ]:
r = mdx.get_current_project()
print(json.dumps(r, indent=2))

### プロジェクト操作履歴の取得

In [ ]:
r = mdx.get_project_history()
print(json.dumps(r, indent=2))

## ネットワーク関連

### ネットワークセグメント一覧情報取得  
プロジェクトで利用可能なネットワークセグメント一覧情報を取得します。

In [ ]:
r = mdx.get_segments()
print(json.dumps(r, indent=2, ensure_ascii=False))

### ネットワークセグメントサマリ情報取得  

セグメントIDを指定し、そのネットワークセグメントの詳細情報を取得します。

In [ ]:
# segment_id = ''
r = mdx.get_segment_summary(segment_id)
print(json.dumps(r, indent=2, ensure_ascii=False))

### ACLリスト取得(IPv4)  

セグメントIDを指定し、許可済みのACLリストを取得します。

In [ ]:
# segment_id = ''
r = mdx.get_allow_acl_ipv4_info(segment_id)
print(json.dumps(r, indent=2, ensure_ascii=False))

### ACLリスト追加(IPv4)  

ACL情報を追加します。

In [ ]:
# acl_ipv4_spec = {
#             "segment": ネットワークセグメントID
#             "src_address": Src IPv4アドレス
#             "src_mask": Srcマスク
#             "src_port": Srcポート
#             "dst_address": Dst IPv4アドレス
#             "dst_mask": Dstマスクの文字列表現
#             "dst_port": Dstポートの文字列表現
#             "protocol": プロトコル "ICMP" "TCP" "UDP" のいずれか
#         }

mdx.add_allow_acl_ipv4_info(acl_ipv4_spec)

### ACL情報削除  

指定したACL情報を削除します。

In [ ]:
# acl_ipv4_id = ''
mdx.delete_allow_acl_ipv4_info(acl_ipv4_id)

### 未使用グローバルIPアドレスリスト取得

DNAT設定で紐づけされていない、未使用のグローバルIPアドレス一覧を取得します。

In [ ]:
r = mdx.get_assignable_global_ipv4()
print(json.dumps(r, indent=2, ensure_ascii=False))

### プロジェクトに属するDNAT情報の取得  

In [ ]:
r = mdx.get_dnat()
print(json.dumps(r, indent=2, ensure_ascii=False))

### 指定したセグメントへのDNAT追加

In [ ]:
# dnat_spec = {
#                 "pool_address": 転送元グローバルIPv4アドレス。プロジェクトの未使用グローバルIPアドレスを指定する。
#                 "segment": ネットワークセグメントID
#                 "dst_address": 転送先プライベートIPアドレス。セグメントIPアドレス範囲内のIPアドレスを指定する。
#             }
mdx.add_dnat(dnat_spec)

### DNAT情報の編集

In [ ]:
# dnat_id = ''
# dnat_spec = {
#                 "pool_address": 転送元グローバルIPv4アドレス。プロジェクトの未使用グローバルIPアドレスを指定する。
#                 "segment": ネットワークセグメントID
#                 "dst_address": 転送先プライベートIPアドレス。セグメントIPアドレス範囲内のIPアドレスを指定する。
#             }
mdx.edit_dnat(dnat_id, dnat_spec)

### DNAT情報の削除

In [ ]:
# dnat_id = ''
mdx.delete_dnat(dnat_id)

## 仮想マシン関連

仮想マシンのデプロイ等に関するライブラリ一覧

### 仮想マシンデプロイカタログ取得

In [ ]:
import json

r = mdx.get_vm_catalogs()
print(json.dumps(r['results'], indent=2))

### 仮想マシンデプロイ

SSHログインに利用するために、仮想マシンデプロイ時に登録する公開鍵を指定します。

In [ ]:
ssh_public_key_path = "~/.ssh/id_ed25519.pub"
with open(os.path.expanduser(ssh_public_key_path)) as f:
    ssh_shared_key = f.read()

- 以下の例では、仮想マシンテンプレートとして「推奨版、東京大学制作、20220412-2043版」を使用する。
  * 利用可能な仮想マシンテンプレートの一覧は、 `get_vm_catalogs()` により確認できる。
- [ハードウェアのカスタマイズ項目](https://docs.mdx.jp/ja/main/create_vm.html#deploy-settings)は、以下の内容で設定する。
  * 仮想マシン名: `"rest-client-example_0001"` **（プロジェクト内で重複しないこと）**
  * パックタイプ: `"cpu"`
  * パック数: `3` **（「推奨版、東京大学制作、20220412-2043版」の下限が3となっている）**
  * 仮想ディスク(GB): `40` **（「推奨版、東京大学制作、20220412-2043版」の下限が40となっている）**
  * ストレージネットワーク: `"portgroup"`
  * サービスレベル (スポット仮想マシン `spot` または起動保証仮想マシン `guarantee`) : `"spot"`
  * ネットワークセグメントID: `get_segments()`で確認したネットワークセグメント一覧から対象のセグメントのIDを確認し、指定する。
  * ssh_shared_key: 対象のVMへのSSHログインで使用する公開鍵を指定する。
- 戻り値
  * 起動したVMの情報（付与されたプライベートIPアドレス等）

In [ ]:
DEFAULT_CATALOG = "16a41081-a1cf-428e-90d0-a147b3aa6fc2"
DEFAULT_TEMPLATE_NAME = "UT-20220412-2043-ubuntu-2004-server"

vm_name = "rest-client-example_0001"

mdx_spec = dict(
    catalog=DEFAULT_CATALOG,
    template_name=DEFAULT_TEMPLATE_NAME,
    pack_num=3,
    pack_type="cpu",
    disk_size=40,
    gpu="0",
    network_adapters=[
        dict(
            adapter_number=1,
            segment=segment_id
        )
    ],
    shared_key=ssh_shared_key,
    storage_network="portgroup",
    service_level="spot",
)

info = mdx.deploy_vm(vm_name, mdx_spec)

print(json.dumps(info, indent=2))

### 仮想マシン一覧取得  
デプロイ済みの仮想マシン一覧を取得します。

In [ ]:
r = mdx.get_vm_list()
print(json.dumps(r, indent=2, ensure_ascii=False))

### 仮想マシン情報取得  
仮想マシン名で対象を指定します。

In [ ]:
r = mdx.get_vm_info("rest-client-example_0001")
print(json.dumps(r, indent=2, ensure_ascii=False))

### 仮想マシンの再起動  
仮想マシン名で対象を指定します。

In [ ]:
mdx.reboot_vm(vm_name, wait_for=True)

### 仮想マシンのシャットダウン  
仮想マシン名で対象を指定します。

In [ ]:
mdx.power_shutdown_vm(vm_name, wait_for=True)

### 仮想マシンの強制停止  
仮想マシン名で対象を指定します。

In [ ]:
mdx.power_off_vm(vm_name, wait_for=True)

### 仮想マシンの起動  
仮想マシン名で対象を指定します。

In [ ]:
mdx.power_on_vm(vm_name, service_level="spot", wait_for=True)

### 仮想マシンの削除  
仮想マシン名で対象を指定します。  
仮想マシンが稼働状態では実行できません。仮想マシンを停止してから実行してください。

In [ ]:
mdx.destroy_vm(vm_name, wait_for=True)